<a href="https://colab.research.google.com/github/JdeGraftJohnson/Python-R-Portfolio/blob/main/bart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
"""
Financial Prediction Model using PyMC-BART on Google Colab

This script implements a hybrid model that combines neural network embeddings
with Bayesian Additive Regression Trees for financial time series forecasting
with uncertainty quantification.

Optimized for Google Colab GPU environment.
"""

# Required packages are already installed according to your output

import os
import torch
import re
import glob
from typing import Dict, List, Optional, Tuple, Union
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from typing import Optional, List, Dict, Union
import pymc as pm
import pymc_bart as pmb
import arviz as az
import matplotlib.pyplot as plt
from pymc_bart.split_rules import SplitRule
import pymc_bart as pmb

# Check for available hardware acceleration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device for PyTorch: {device}")
print("Using device for PyMC: CPU (default)")

# Setup for data storage - try Google Drive first, with fallback to local Colab storage
USE_DRIVE = False
DRIVE_PATH = '/content/drive/MyDrive/financial_bart_model'
LOCAL_PATH = '/content/financial_bart_model'

try:
    from google.colab import drive, files
    # Try to mount Google Drive
    drive.mount('/content/drive')
    # Create directory for saving models and results if it doesn't exist
    os.makedirs(DRIVE_PATH, exist_ok=True)
    print(f"Successfully mounted Google Drive. Using path: {DRIVE_PATH}")
    USE_DRIVE = True
except Exception as e:
    print(f"Could not mount Google Drive: {str(e)}")
    print(f"Using local Colab storage at: {LOCAL_PATH}")
    os.makedirs(LOCAL_PATH, exist_ok=True)

# Set the storage path based on Drive availability
STORAGE_PATH = DRIVE_PATH if USE_DRIVE else LOCAL_PATH

# Constants for model configuration
EMBEDDING_DIM = 768  # Standard dimension for all embeddings
FUSION_OUTPUT_DIM = 128  # Output dimension after fusion

class EmbeddingProcessor(nn.Module):
    """
    Processes embeddings from multiple financial data sources
    and fuses them for downstream Bayesian modeling
    """
    def __init__(self,
                embedding_dim=EMBEDDING_DIM,
                output_dim=FUSION_OUTPUT_DIM):
        super().__init__()

        # All projections use the same embedding dimension for consistency
        self.stock_projection = nn.Sequential(
            nn.Linear(embedding_dim, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Linear(256, output_dim)
        )

        self.sentiment_projection = nn.Sequential(
            nn.Linear(embedding_dim, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Linear(256, output_dim)
        )

        # Feature fusion layer
        self.fusion = nn.Sequential(
            nn.Linear(2 * output_dim, output_dim),
            nn.LayerNorm(output_dim),
            nn.ReLU()
        )

        # Move model to appropriate device
        self.to(device)

    def extract_features(self, embeds, projection_layer):
        """Extract features using the specified projection layer"""
        # Handle both 2D and 3D inputs consistently
        if len(embeds.shape) == 3:
            embeds = torch.mean(embeds, dim=1)
        return projection_layer(embeds)

    def forward(self, stock_embeds, sentiment_embeds):
        """
        Forward pass through the embedding processor

        Args:
            stock_embeds: Stock data embeddings [batch_size, embedding_dim]
            sentiment_embeds: Sentiment data embeddings [batch_size, embedding_dim]

        Returns:
            Processed features for Bayesian modeling [batch_size, output_dim]
        """
        # Move inputs to the correct device
        stock_embeds = stock_embeds.to(device)
        sentiment_embeds = sentiment_embeds.to(device)

        # Project each data source
        stock_features = self.extract_features(stock_embeds, self.stock_projection)
        sentiment_features = self.extract_features(sentiment_embeds, self.sentiment_projection)

        # Concatenate features
        combined = torch.cat([stock_features, sentiment_features], dim=1)

        # Fuse features
        fused = self.fusion(combined)

        return fused


class PyMcBartPredictionHead:
    """BART model with PyMC-BART prediction head for financial forecasting with uncertainty."""
    def __init__(
        self,
        n_trees=50,
        alpha=0.90,  # Slightly reduced from 0.95 to allow more flexibility
        beta=1.5,    # Slightly reduced from 2.0 to allow more flexibility
        num_classes=None,  # Set for classification tasks
        response_type="continuous"  # 'continuous', 'binary', or 'categorical'
    ):
        self.n_trees = n_trees
        self.alpha = alpha
        self.beta = beta
        self.num_classes = num_classes
        self.response_type = response_type
        self.model = None
        self.idata = None  # To store inference data
        self.is_fitted = False

    def _build_model(self, X, y=None):
        """Build the PyMC-BART model."""
        with pm.Model() as model:
            # Define BART model based on response type
            if self.response_type == "continuous":
                # Regression model without jitter (which caused the error)
                μ = pmb.BART(
                    "μ",
                    X,
                    y,
                    m=self.n_trees,
                    alpha=self.alpha,
                    beta=self.beta
                )

                # Add likelihood
                σ = pm.HalfNormal("σ", 1.0)
                y_pred = pm.Normal("y_pred", mu=μ, sigma=σ, observed=y)

            elif self.response_type == "binary":
                # Binary classification model
                μ = pmb.BART(
                    "μ",
                    X,
                    y,
                    m=self.n_trees,
                    alpha=self.alpha,
                    beta=self.beta
                )

                # Add sigmoid link and likelihood
                p = pm.Deterministic("p", pm.math.sigmoid(μ))
                y_pred = pm.Bernoulli("y_pred", p=p, observed=y)

            elif self.response_type == "categorical":
                # Ensure we have the proper dimensions for multiclass classification
                if self.num_classes is None:
                    raise ValueError("For categorical response, num_classes must be specified")

                # Set up coordinates for multiple outputs
                coords = {"classes": range(self.num_classes), "obs_id": range(len(X))}

                with pm.Model(coords=coords) as model:
                    # For categorical, we need one BART model per class
                    μ = pmb.BART(
                        "μ",
                        X,
                        y,
                        m=self.n_trees,
                        alpha=self.alpha,
                        beta=self.beta,
                        dims=["classes", "obs_id"]
                    )

                    # Apply softmax to get probabilities
                    θ = pm.Deterministic("θ", pm.math.softmax(μ, axis=0))

                    # Categorical likelihood
                    y_pred = pm.Categorical("y_pred", p=θ.T, observed=y)

            else:
                raise ValueError(f"Unsupported response type: {self.response_type}")

        return model

    def fit(
        self,
        X,
        y,
        chains=4,  # Increased to 4 chains for better uncertainty estimates
        tune=1000,
        draws=1000,
        cores=2,  # Use multiple cores on Colab
        random_seed=None
    ):
        """
        Fit the PyMC-BART model.

        Args:
            X: Feature matrix
            y: Target values
            chains: Number of MCMC chains
            tune: Number of tuning samples
            draws: Number of posterior samples
            cores: Number of cores to use
            random_seed: Random seed for reproducibility

        Returns:
            self
        """
        # Build the model
        self.model = self._build_model(X, y)

        # Sample from the posterior
        with self.model:
            self.idata = pm.sample(
                draws=draws,
                tune=tune,
                chains=chains,
                cores=cores,
                random_seed=random_seed,
                compute_convergence_checks=True
            )

            # Get posterior predictive samples
            pm.sample_posterior_predictive(
                self.idata,
                extend_inferencedata=True
            )

        self.is_fitted = True
        return self

    def predict(self, X, return_std=False, samples=100):
        """
        Make predictions with uncertainty estimates.
        """
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before making predictions")

        # Get posterior samples for μ
        μ_samples = self.idata.posterior["μ"].values

        # Debug information
        print("Original μ_samples shape:", μ_samples.shape)
        print("Sample values from μ_samples:", μ_samples[0, 0, :5])  # Print first 5 values from first chain/draw

        # Properly reshape samples from multiple chains
        # Reshape to (chains*draws, observations)
        μ_samples = μ_samples.reshape(-1, μ_samples.shape[-1])

        # Take a subset of samples if needed
        if samples < μ_samples.shape[0]:
            μ_samples = μ_samples[:samples]

        print("Reshaped μ_samples shape:", μ_samples.shape)

        # For binary classification, apply sigmoid
        if self.response_type == "binary":
            μ_samples = 1.0 / (1.0 + np.exp(-μ_samples))

        # For categorical classification, apply softmax
        elif self.response_type == "categorical":
            μ_samples = np.transpose(μ_samples, (0, 2, 1))
            exp_scores = np.exp(μ_samples)
            μ_samples = exp_scores / np.sum(exp_scores, axis=2, keepdims=True)

        # Check for variation across samples
        sample_variation = np.var(μ_samples, axis=0)
        print("Variance across samples (first 5):", sample_variation[:5])
        print("Min/max/mean variance:", np.min(sample_variation), np.max(sample_variation), np.mean(sample_variation))

        # Compute mean predictions
        predictions = np.mean(μ_samples, axis=0)

        if return_std:
            # Compute standard deviation of predictions with small epsilon to avoid zeros
            std_predictions = np.std(μ_samples, axis=0)
            print("Raw std calculations (first 5):", std_predictions[:5])

            # Ensure uncertainties aren't exactly zero by adding a small epsilon
            epsilon = 1e-8
            std_predictions = np.maximum(std_predictions, epsilon)

            return predictions, std_predictions

        return predictions

    def get_variable_importance(self, X, labels=None):
        """
        Get variable importance metrics from the fitted model.

        Args:
            X: Feature matrix
            labels: List of feature names

        Returns:
            Dictionary with variable importance results
        """
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before computing variable importance")

        # Fix for the labels input based on NumPy docs
        if labels is not None:
            if isinstance(labels, np.ndarray):
                labels = labels.tolist()  # Convert numpy array to list
            elif isinstance(labels, list):
                # Already a list, keep as is
                pass
            else:
                # Other types, try to convert
                labels = list(labels)

        # Calculate variable importance
        try:
            variable_importance = pmb.get_variable_inclusion(self.idata, X, labels=labels)

            # Compute more detailed variable importance
            vi_results = pmb.compute_variable_importance(
                self.idata,
                self.model.named_vars["μ"],
                X,
                method="VI"
            )

            return {
                "normalized_vi": variable_importance[0],
                "variable_labels": variable_importance[1],
                "detailed_results": vi_results
            }
        except Exception as e:
            print(f"Error in calculating variable importance: {str(e)}")
            print(f"Type of labels: {type(labels)}")
            if labels is not None:
                print(f"First few labels: {labels[:5] if len(labels) > 5 else labels}")
            raise

    def plot_variable_importance(self, X, labels=None):
        """Plot variable importance metrics."""
        if not self.is_fitted:
            raise RuntimeError("Model must be fitted before plotting variable importance")

        # Fix for the labels input based on NumPy docs
        if labels is not None:
            if isinstance(labels, np.ndarray):
                labels = labels.tolist()  # Convert numpy array to list
            elif isinstance(labels, list):
                # Already a list, keep as is
                pass
            else:
                # Other types, try to convert
                labels = list(labels)

        return pmb.plot_variable_inclusion(self.idata, X, labels=labels)


class FinancialPredictionModel:
    """
    Hybrid model combining embedding processing with PyMC-BART for financial forecasting
    """
    def __init__(
        self,
        embedding_dim=EMBEDDING_DIM,
        output_dim=FUSION_OUTPUT_DIM,
        n_trees=50,
        response_type="continuous",
        num_classes=None
    ):
        # Feature extractor with consistent embedding dimensions
        self.feature_extractor = EmbeddingProcessor(
            embedding_dim=embedding_dim,
            output_dim=output_dim
        )

        # PyMC-BART prediction head
        self.pymc_bart_head = PyMcBartPredictionHead(
            n_trees=n_trees,
            response_type=response_type,
            num_classes=num_classes
        )

    def extract_features(self, stock_embeds, sentiment_embeds):
        """Extract features from input data"""
        with torch.no_grad():
            features = self.feature_extractor(
                stock_embeds,
                sentiment_embeds
            )
        return features.cpu().numpy()

    def fit(
        self,
        stock_embeds,
        sentiment_embeds,
        labels,
        chains=4,  # Increased for better uncertainty estimates
        tune=1000,
        draws=1000,
        cores=2,  # Use multiple cores on Colab
        random_seed=None
    ):
        """Fit the model to training data"""
        # Extract features
        X = self.extract_features(
            stock_embeds,
            sentiment_embeds
        )

        # Convert labels to numpy if needed
        if isinstance(labels, torch.Tensor):
            labels = labels.cpu().numpy()

        # Fit PyMC-BART head
        self.pymc_bart_head.fit(
            X,
            labels,
            chains=chains,
            tune=tune,
            draws=draws,
            cores=cores,
            random_seed=random_seed
        )

        return self

    def predict(
        self,
        stock_embeds,
        sentiment_embeds,
        return_std=False,
        samples=100
    ):
        """Make predictions with uncertainty estimates"""
        # Extract features
        X = self.extract_features(
            stock_embeds,
            sentiment_embeds
        )

        # Make predictions
        return self.pymc_bart_head.predict(X, return_std=return_std, samples=samples)

    def get_variable_importance(self, stock_embeds, sentiment_embeds, feature_names=None):
        """Get variable importance metrics"""
        # Extract features
        X = self.extract_features(
            stock_embeds,
            sentiment_embeds
        )

        # Fix for the labels input - convert properly for pymc_bart
        if feature_names is not None:
            if isinstance(feature_names, np.ndarray):
                feature_names = feature_names.tolist()
            elif isinstance(feature_names, list):
                # Keep as is
                pass
            else:
                # Convert other types if necessary
                feature_names = list(feature_names)

        # Pass to the PyMcBartPredictionHead
        return self.pymc_bart_head.get_variable_importance(X, labels=feature_names)

    def plot_variable_importance(self, stock_embeds, sentiment_embeds, feature_names=None):
        """Plot variable importance"""
        X = self.extract_features(
            stock_embeds,
            sentiment_embeds
        )

        # Fix for the labels input - convert properly for pymc_bart
        if feature_names is not None:
            if isinstance(feature_names, np.ndarray):
                feature_names = feature_names.tolist()
            elif isinstance(feature_names, list):
                # Keep as is
                pass
            else:
                # Convert other types if necessary
                feature_names = list(feature_names)

        return self.pymc_bart_head.plot_variable_importance(X, labels=feature_names)

    def save_model(self, path=None):
        """Save model to disk"""
        if path is None:
            path = os.path.join(DRIVE_PATH, "financial_model_traces.nc")

        if not self.pymc_bart_head.is_fitted:
            raise RuntimeError("Model must be fitted before saving")

        # Save PyMC-BART traces
        self.pymc_bart_head.idata.to_netcdf(path)
        print(f"Model traces saved to {path}")

        # Save PyTorch feature extractor
        torch_path = os.path.join(os.path.dirname(path), "feature_extractor.pt")
        torch.save(self.feature_extractor.state_dict(), torch_path)
        print(f"Feature extractor saved to {torch_path}")

        return path

    def load_model(self, path=None, trace_path=None, extractor_path=None):
        """Load model from disk"""
        if path is not None:
            # Use path as base directory
            trace_path = path if trace_path is None else trace_path
            extractor_path = os.path.join(os.path.dirname(path), "feature_extractor.pt") if extractor_path is None else extractor_path
        elif trace_path is None:
            trace_path = os.path.join(DRIVE_PATH, "financial_model_traces.nc")
            extractor_path = os.path.join(DRIVE_PATH, "feature_extractor.pt")

        # Load PyMC-BART traces
        try:
            self.pymc_bart_head.idata = az.from_netcdf(trace_path)
            print(f"Model traces loaded from {trace_path}")

            # Load PyTorch feature extractor if it exists
            if os.path.exists(extractor_path):
                self.feature_extractor.load_state_dict(torch.load(extractor_path))
                print(f"Feature extractor loaded from {extractor_path}")

            self.pymc_bart_head.is_fitted = True
            return True
        except Exception as e:
            print(f"Error loading model: {str(e)}")
            return False


def load_and_preprocess_data(stock_data_path, sentiment_data_path, max_rows=None):
    print(f"Loading data from: {stock_data_path}, {sentiment_data_path}")

    stock_df = pd.read_csv(stock_data_path)
    sentiment_df = pd.read_csv(sentiment_data_path)

    min_rows = min(len(stock_df), len(sentiment_df))

    if max_rows is not None:
        min_rows = min(min_rows, max_rows)

    print(f"Processing {min_rows} rows")

    stock_embeds = _extract_embeddings(stock_df, min_rows)
    sentiment_embeds = _extract_sentiment_embeddings(sentiment_df, min_rows)
    labels = _extract_labels(stock_df, min_rows)

    return {
        "stock_embeds": stock_embeds,
        "sentiment_embeds": sentiment_embeds,
        "labels": labels,
        "num_samples": min_rows
    }

def _extract_embeddings(df, rows):
    embed_cols = [col for col in df.columns if col.startswith('gpt_embed_')]

    if embed_cols:
        return torch.tensor(df[embed_cols].values[:rows], dtype=torch.float32)
    else:
        return torch.randn(rows, EMBEDDING_DIM)

def _extract_sentiment_embeddings(df, rows):
    sentiment_score_cols = ['sentiment_compound', 'sentiment_positive',
                             'sentiment_neutral', 'sentiment_negative']

    available_cols = [col for col in sentiment_score_cols if col in df.columns]

    if available_cols:
        base_features = torch.tensor(df[available_cols].values[:rows], dtype=torch.float32)
        projection = nn.Sequential(
            nn.Linear(len(available_cols), 128),
            nn.ReLU(),
            nn.Linear(128, EMBEDDING_DIM)
        )

        with torch.no_grad():
            return projection(base_features)
    else:
        return torch.randn(rows, EMBEDDING_DIM)

def _extract_labels(df, rows):
    return torch.tensor(
        df['target'].values[:rows] if 'target' in df.columns
        else np.random.randn(rows),
        dtype=torch.float32
    ).unsqueeze(1)

def plot_predictions_with_uncertainty(predictions, uncertainties, save_path=None):
    """
    Plot predictions with uncertainty bands
    """
    plt.figure(figsize=(14, 6))

    x = np.arange(len(predictions))
    plt.plot(x, predictions, 'b-', label='Predictions')

    # Plot uncertainty bands
    plt.fill_between(
        x,
        predictions - uncertainties,
        predictions + uncertainties,
        alpha=0.3, color='b', label='Uncertainty'
    )

    plt.title('Financial Predictions with Uncertainty', fontsize=16)
    plt.xlabel('Time', fontsize=12)
    plt.ylabel('Value', fontsize=12)
    plt.legend()
    plt.grid(True)

    if save_path:
        plt.savefig(save_path)
        print(f"Plot saved to {save_path}")

    plt.show()


if __name__ == "__main__":
    # Upload datasets to Colab (if not already in Drive)
    try:
        stock_data_path = os.path.join(DRIVE_PATH, "stock_data_with_embeddings.csv")
        sentiment_data_path = os.path.join(DRIVE_PATH, "sentiment_data_with_embeddings.csv")
        # Add after saving each file:
        if os.path.exists(file_path):
            print(f"File successfully saved to {file_path} (Size: {os.path.getsize(file_path)} bytes)")
        else:
            print(f"ERROR: Failed to save file to {file_path}")
        # Check if files exist in Drive, if not, ask for upload
        if not os.path.exists(stock_data_path) or not os.path.exists(sentiment_data_path):
            print("Please upload the stock and sentiment data CSV files")
            uploaded = files.upload()
            # Add after saving each file:
            if os.path.exists(file_path):
                print(f"File successfully saved to {file_path} (Size: {os.path.getsize(file_path)} bytes)")
            else:
                print(f"ERROR: Failed to save file to {file_path}")

            # Move uploaded files to Drive
            for filename in uploaded.keys():
                if "stock" in filename.lower():
                    os.replace(filename, stock_data_path)
                    print(f"Moved {filename} to {stock_data_path}")
                elif "sentiment" in filename.lower():
                    os.replace(filename, sentiment_data_path)
                    print(f"Moved {filename} to {sentiment_data_path}")
    except Exception as e:
        print(f"Error managing file upload: {str(e)}")
        # Fallback paths if there's an issue with Drive
        stock_data_path = "stock_data_with_embeddings.csv"
        sentiment_data_path = "sentiment_data_with_embeddings.csv"

    # Create and train the model
    model = FinancialPredictionModel(
        embedding_dim=EMBEDDING_DIM,
        output_dim=FUSION_OUTPUT_DIM,
        n_trees=50,
        response_type="continuous"
    )

    # Load data with consistent dimensions
    data = load_and_preprocess_data(
        stock_data_path=stock_data_path,
        sentiment_data_path=sentiment_data_path
    )

    # Extract items from data
    stock_embeds = data["stock_embeds"]
    sentiment_embeds = data["sentiment_embeds"]
    labels = data["labels"]

    # Log dimensions to verify consistency
    print(f"Stock embeddings shape: {stock_embeds.shape}")
    print(f"Sentiment embeddings shape: {sentiment_embeds.shape}")
    print(f"Labels shape: {labels.shape}")

    # Train the model with multiple chains for better uncertainty estimates
    print("Fitting the model...")
    model.fit(
        stock_embeds=stock_embeds,
        sentiment_embeds=sentiment_embeds,
        labels=labels.squeeze(),
        chains=4,  # Multiple chains for better uncertainty estimates
        tune=500,  # More tuning steps
        draws=500,  # More draws
        cores=2,   # Use multiple cores on Colab
        random_seed=123
    )

    # Make predictions
    print("Making predictions...")
    predictions, uncertainties = model.predict(
        stock_embeds=stock_embeds,
        sentiment_embeds=sentiment_embeds,
        return_std=True
    )

    print(f"Predictions shape: {predictions.shape}")
    print(f"Uncertainties shape: {uncertainties.shape}")

    # Print uncertainty statistics
    print("Sample uncertainties:", uncertainties.flatten()[:10])
    print("Min uncertainty:", np.min(uncertainties))
    print("Max uncertainty:", np.max(uncertainties))
    print("Mean uncertainty:", np.mean(uncertainties))

    # Plot predictions with uncertainties
    plot_predictions_with_uncertainty(
        predictions.flatten(),
        uncertainties.flatten(),
        save_path=os.path.join(DRIVE_PATH, "predictions_plot.png")
    )

    # Calculate variable importance with better error handling
    print("Calculating variable importance...")
    feature_names = [f"Feature_{i}" for i in range(FUSION_OUTPUT_DIM)]

    try:
        # First try plotting convergence as recommended in the docs
        pmb.plot_convergence(model.pymc_bart_head.idata, var_name="μ")
        plt.savefig(os.path.join(DRIVE_PATH, "convergence_plot.png"))
        plt.show()

        # Try safer variable importance calculation
        importance = model.get_variable_importance(
            stock_embeds=stock_embeds,
            sentiment_embeds=sentiment_embeds,
            feature_names=feature_names
        )
        print("Variable importance calculation successful!")

        # Try plotting variable importance
        try:
            vi_results = pmb.compute_variable_importance(
                model.pymc_bart_head.idata,
                model.pymc_bart_head.model.named_vars["μ"],
                model.extract_features(stock_embeds, sentiment_embeds)
            )
            plt.figure(figsize=(10, 6))
            pmb.plot_variable_importance(vi_results)
            plt.savefig(os.path.join(DRIVE_PATH, "variable_importance.png"))
            plt.show()
        except Exception as e:
            print(f"Could not plot variable importance: {str(e)}")

    except Exception as e:
        print(f"Error calculating variable importance: {str(e)}")
        print("Variable importance is not crucial for BART models; continuing with predictions")

    # Save the model traces and feature extractor
    print("Saving model...")
    model.save_model()

    # Save predictions to CSV

    print("Saving predictions to CSV...")
    results_df = pd.DataFrame({
        "predictions": predictions.flatten(),
        "uncertainties": uncertainties.flatten()
    })
# Add after saving each file:
    if os.path.exists(file_path):
        print(f"File successfully saved to {file_path} (Size: {os.path.getsize(file_path)} bytes)")
    else:
        print(f"ERROR: Failed to save file to {file_path}")
    results_path = os.path.join(DRIVE_PATH, "model_predictions.csv")
    results_df.to_csv(results_path, index=False)
    print(f"Predictions saved to {results_path}")

    # Add partial dependence plots if features are interpretable
    try:
        print("Generating partial dependence plots...")
        # Get the BART variable directly from the model
        bart_var = model.pymc_bart_head.model.named_vars["μ"]
        # Generate feature matrix for PDP
        X_features = model.extract_features(stock_embeds, sentiment_embeds)
        # Create partial dependence plots
        plt.figure(figsize=(12, 10))
        pmb.plot_pdp(bart_var, X=X_features, Y=labels.squeeze().numpy(),
                    grid=(2, 2), func=np.exp)
        plt.savefig(os.path.join(DRIVE_PATH, "partial_dependence_plots.png"))
        plt.show()
        print("Partial dependence plots generated successfully!")
    except Exception as e:
        print(f"Error generating partial dependence plots: {str(e)}")

    print("Training and evaluation complete!")

AttributeError: partially initialized module 'torch' has no attribute 'fx' (most likely due to a circular import)

In [11]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

AttributeError: partially initialized module 'torch' has no attribute 'fx' (most likely due to a circular import)